In [ ]:
# 1. Cài đặt thư viện cần thiết
!pip install pydub

import os
import shutil
import librosa
import soundfile as sf
import hashlib
import pandas as pd
from tqdm import tqdm
from pydub import AudioSegment
from google.colab import files


In [ ]:
from google.colab import files
import os

# Tạo thư mục uploads nếu chưa có
upload_dir = 'uploads'
os.makedirs(upload_dir, exist_ok=True)

# Upload file
uploaded = files.upload()

# Lưu vào thư mục uploads
for filename in uploaded.keys():
    os.rename(filename, os.path.join(upload_dir, filename))


Saving common_voice_vi_21824030 (1).wav to common_voice_vi_21824030 (1) (1).wav
Saving common_voice_vi_21824030.wav to common_voice_vi_21824030 (2).wav
Saving common_voice_vi_21824032 (1).wav to common_voice_vi_21824032 (1) (1).wav
Saving common_voice_vi_21824032.wav to common_voice_vi_21824032 (2).wav
Saving common_voice_vi_21824033 (1).wav to common_voice_vi_21824033 (1) (1).wav
Saving common_voice_vi_21824033.wav to common_voice_vi_21824033 (2).wav
Saving common_voice_vi_21824034 (1).wav to common_voice_vi_21824034 (1) (1).wav
Saving common_voice_vi_21824034.wav to common_voice_vi_21824034 (2).wav
Saving common_voice_vi_21824045 (1).wav to common_voice_vi_21824045 (1) (1).wav
Saving common_voice_vi_21824045.wav to common_voice_vi_21824045 (2).wav
Saving common_voice_vi_21824047 (1).wav to common_voice_vi_21824047 (1) (1).wav
Saving common_voice_vi_21824047.wav to common_voice_vi_21824047 (2).wav
Saving common_voice_vi_21824049 (1).wav to common_voice_vi_21824049 (1) (1).wav
Saving c

In [ ]:
import hashlib
from pydub import AudioSegment
import os
import csv
from tqdm import tqdm

output_dir = 'processed'
os.makedirs(output_dir, exist_ok=True)

# Hàm tạo hash từ nội dung file
def get_file_hash(file_path):
    with open(file_path, 'rb') as f:
        return hashlib.md5(f.read()).hexdigest()

# Duyệt file và lọc trùng
seen_hashes = set()
metadata = []
count = 0

all_files = [f for f in os.listdir(upload_dir) if f.lower().endswith('.wav')]

for i, fname in enumerate(tqdm(all_files)):
    path = os.path.join(upload_dir, fname)

    try:
        file_hash = get_file_hash(path)
        if file_hash in seen_hashes:
            continue

        seen_hashes.add(file_hash)

        audio = AudioSegment.from_wav(path)

        original_duration = round(audio.duration_seconds, 2)
        sample_rate = audio.frame_rate
        channels = audio.channels
        size_kb = round(os.path.getsize(path) / 1024, 2)

        # Đổi tên file đã xử lý
        new_fname = f"audio_{count+1:03d}.wav"
        new_path = os.path.join(output_dir, new_fname)
        audio.export(new_path, format='wav')

        # Ghi metadata
        metadata.append({
            'STT': count + 1,
            'Tên file gốc': fname,
            'Tên file WAV sau xử lý': new_fname,
            'Thời lượng gốc (giây)': original_duration,
            'Thời lượng sau xử lý (giây)': original_duration,
            'Sample Rate (Hz)': sample_rate,
            'Số kênh': channels,
            'Kích thước WAV (KB)': size_kb
        })

        count += 1

    except Exception as e:
        print(f"Lỗi xử lý {fname}: {e}")

# Giữ lại đúng 300 file
if len(metadata) > 300:
    metadata = metadata[:300]
    for m in metadata:
        pass  # đã giữ nguyên
    # Xóa các file dư
    all_output_files = sorted(os.listdir(output_dir))
    for extra in all_output_files[300:]:
        os.remove(os.path.join(output_dir, extra))


100%|██████████| 608/608 [00:00<00:00, 1844.20it/s]


In [ ]:
import pandas as pd

df = pd.DataFrame(metadata)
csv_path = os.path.join(output_dir, 'metadata.csv')
df.to_csv(csv_path, index=False, encoding='utf-8-sig')


In [ ]:
import shutil

shutil.make_archive('processed_audio', 'zip', output_dir)

files.download('processed_audio.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>